In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config('spark.driver.maxResultSize', '18g').getOrCreate()

In [0]:
from pyspark.sql.functions import lit

In [0]:
import pyspark.sql.functions as F

In [0]:
def handler(sparkdf):
    df = sparkdf.toPandas()
    filtered_df = df[df["title"].str.contains("ukraine|russia|russian|ukraina|putin|kyiv|moscow|invasion|vladimir")==True]
    #filtered_df['sub_time']=subtime
    return spark.createDataFrame(filtered_df) 

In [0]:
def read_dataset(year_range = [2015, 2016, 2017], month_range = range(1, 13), comments_or_submissions = "submissions"):
    if comments_or_submissions == "comments":
        link = "comments/RC_"
    elif comments_or_submissions == "submissions":
        link = "submissions/RS_"
    else:
        link = "non-exist"
    result = None
    for year in year_range:
        for month in month_range:
            month_literal = ""
            if month < 10:
                month_literal = "0" + str(month)
            else:
                month_literal = str(month)
            data_path = "/mnt/lsde/datasets/reddit/" + link + str(year) + "-" + month_literal + ".json.bz2"
            subtime=str(year) + "-" + month_literal
            substrings ="ukraine|russia|russian|ukraina|putin|kyiv|moscow|invasion|vladimir"
            
            if result is None:
                print("handling"+str(year) + "-" + month_literal )
                #result = handler(spark.read.load(data_path,format="json").filter("author!='[deleted]'").filter("title!='[deleted]'").select("author","title","subreddit","subreddit_id","id").withColumn("subtime",lit(subtime)))
                result =spark.read.option("samplingRatio",0.000001).load(data_path,format="json").filter("author!='[deleted]'").filter("title!='[deleted]'").select("author","title","subreddit","subreddit_id","id")
                result = result.filter(F.col("title").rlike(substrings)).withColumn("subtime",lit(subtime))
            else:
                print("handling"+str(year) + "-" + month_literal )
                tmp = spark.read.option("samplingRatio",0.000001).load(data_path,format="json").filter("author!='[deleted]'").filter("title!='[deleted]'").select("author","title","subreddit","subreddit_id","id")
                tmp = tmp.filter(F.col("title").rlike(substrings)).withColumn("subtime",lit(subtime))
                result = result.union(tmp)
    result.write.mode("overwrite").format("parquet").save("/mnt/lsde/group05/"+ comments_or_submissions+"/"+str(year) +".parquet")

    return result

In [0]:
def read_dataset_withpath(year_range = [2015, 2016, 2017], month_range = range(1, 13), comments_or_submissions = "submissions",path=""):
    if comments_or_submissions == "comments":
        link = "comments/RC_"
    elif comments_or_submissions == "submissions":
        link = "submissions/RS_"
    else:
        link = "non-exist"
    result = None
    for year in year_range:
        for month in month_range:
            month_literal = ""
            if month < 10:
                month_literal = "0" + str(month)
            else:
                month_literal = str(month)
            data_path = "/mnt/lsde/datasets/reddit/" + link + str(year) + "-" + month_literal + ".json.bz2"
            subtime=str(year) + "-" + month_literal
            substrings ="ukraine|russia|russian|ukraina|putin|kyiv|moscow|invasion|vladimir"
            
            if result is None:
                print("handling"+str(year) + "-" + month_literal )
                #result = handler(spark.read.load(data_path,format="json").filter("author!='[deleted]'").filter("title!='[deleted]'").select("author","title","subreddit","subreddit_id","id").withColumn("subtime",lit(subtime)))
                result = spark.read.load(data_path,format="json").filter("author!='[deleted]'").filter("title!='[deleted]'").select("author","title","subreddit","subreddit_id","id").withColumn("subtime",lit(subtime))
                result = result.filter(F.col("title").rlike(substrings))
            else:
                print("handling"+str(year) + "-" + month_literal )
                tmp = spark.read.load(data_path,format="json").filter("author!='[deleted]'").filter("title!='[deleted]'").select("author","title","subreddit","subreddit_id","id").withColumn("subtime",lit(subtime))
                tmp = tmp.filter(F.col("title").rlike(substrings))
                result = result.union(tmp)
    result.write.mode("overwrite").format("parquet").save("/mnt/lsde/group05/"+ comments_or_submissions+"/"+path +".parquet")

    return result

In [0]:
sub_2014= read_dataset([2014],range(2,13),"submissions")

handling2014-02
handling2014-03
handling2014-04
handling2014-05
handling2014-06
handling2014-07
handling2014-08
handling2014-09
handling2014-10
handling2014-11
handling2014-12

In [0]:
sub_2015= read_dataset([2015],range(1,13),"submissions")

handling2015-01
handling2015-02
handling2015-03
handling2015-04
handling2015-05
handling2015-06
handling2015-07
handling2015-08
handling2015-09
handling2015-10
handling2015-11
handling2015-12

In [0]:
sub_2016= read_dataset([2016],range(1,13),"submissions")

handling2016-01
handling2016-02
handling2016-03
handling2016-04
handling2016-05
handling2016-06
handling2016-07
handling2016-08
handling2016-09
handling2016-10
handling2016-11
handling2016-12

In [0]:
sub_2017= read_dataset([2017],range(1,13),"submissions")

handling2017-01
handling2017-02
handling2017-03
handling2017-04
handling2017-05
handling2017-06
handling2017-07
handling2017-08
handling2017-09
handling2017-10
handling2017-11
handling2017-12

In [0]:
sub_2018= read_dataset([2018],range(1,13),"submissions")

handling2018-01
handling2018-02
handling2018-03
handling2018-04
handling2018-05
handling2018-06
handling2018-07
handling2018-08
handling2018-09
handling2018-10
handling2018-11
handling2018-12

In [0]:
sub_2019= read_dataset([2019],range(1,13),"submissions")

handling2019-01
handling2019-02
handling2019-03
handling2019-04
handling2019-05
handling2019-06
handling2019-07
handling2019-08
handling2019-09
handling2019-10
handling2019-11
handling2019-12

In [0]:
sub_2019_02= read_dataset_withpath([2019],range(2,3),"submissions","2019-02")

In [0]:
sub_2020= read_dataset([2020],range(1,13),"submissions")

handling2020-01
handling2020-02
handling2020-03
handling2020-04
handling2020-05
handling2020-06
handling2020-07
handling2020-08
handling2020-09
handling2020-10
handling2020-11
handling2020-12

In [0]:
 sub_2021=  read_dataset([2021],range(1,13),"submissions")

handling2021-01
handling2021-02
handling2021-03
handling2021-04
handling2021-05
handling2021-06
handling2021-07
handling2021-08
handling2021-09
handling2021-10
handling2021-11
handling2021-12

In [0]:
sub_2022 = read_dataset([2022],range(1,9),"submissions")

handling2022-01
handling2022-02
handling2022-03
handling2022-04
handling2022-05
handling2022-06
handling2022-07
handling2022-08